## Ch5. Resampling methods

Placing most of the imports:

In [1]:
import numpy as np
import statsmodels.api as sm # OLS
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

New imports needed for this lab:

In [2]:
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### The Validation Set Approach

使用 Auto 資料, 將資料隨機分成 training (1/2) + validation (1/2) 

In [3]:
Auto = load_data('Auto') # ISLP 裡面的資料
Auto_train , Auto_valid = train_test_split(Auto ,   #train_test_splint: 來源為 sklern.model_selection
                                        test_size=196,
                                        random_state=0) #random_state=0: 固定 seed

Fit a linear regression using training set: 

In [4]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train) #fit, transform 形成 design matrix, 放入 regression model
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [5]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [6]:
def evalMSE(terms ,    #要放進來的變數名稱
            response , #y 變數名稱
            train ,    #training data
            test):     #test data

    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [7]:
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train ,
                        Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
Auto_train , Auto_valid = train_test_split(Auto ,
                                            test_size=196,
                                            random_state=3)
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train ,
                        Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])